In [1]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize 
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn't guaranteed
gpu = GPUs[0]

def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print('GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB'.format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm()

Gen RAM Free: 11.5 GB  I Proc size: 137.5 MB
GPU RAM Free: 7205MB | Used: 4236MB | Util  37% | Total 11441MB


In [2]:
ls

drive/  sample_data/


In [3]:
cd drive

/content/drive


In [4]:
ls

'My Drive'/


In [5]:
cd My Drive

/content/drive/My Drive


In [6]:
ls

 1stone.h5
 20151122_223055.mp4
'AAA-Literature reviews:.gdoc'
 AAARBI.gdoc
'AAA SOP Progress.gdoc'
'AAA stopwords.gdoc'
'AA BAV Project.gdoc'
'Aa bavv2.gdoc'
 ABC.xlsx
 ans.csv
 ASD.gdoc
 bidirectional42i.h5
 bidirectionalffhai.h5
 bidirectional.h5
 bidirectionalhai.h5
 bidirectionalwithoutglobal.h5
 CNN_Data/
'Colab Notebooks'/
 companies.docx
'Copy of glove.twitter.27B.100d.txt'
'Copy of glove.twitter.27B.50d.txt'
 datarnn.csv
 DESCRIPTION.docx
 DESCRIPTION.gdoc
'dna structure1.ppt'
'DRM Ass 2.2.xlsx'
'DRM Ass 2.2.xlsx.gsheet'
 dt/
 GAIL_Group37_Futures.xlsx
 GAIL_Group37_Futures.xlsx.gsheet
'GAIL_Group37_Underlying Asset.xlsx'
'GAIL_Group37_Underlying Asset.xlsx.gsheet'
 GANs.zip
 GroceryProcessed.csv
 Hatespeech-data/
'Ian Goodfellow, Yoshua Bengio, Aaron Courville-Deep Learning-MIT Press (2016).pdf'
'Ideas and constrains of the strategies.gdoc'
 inc_occ_gender.csv
 instructions_1.pdf
'log returns.xlsx'
 lstmheading.csv
 marketdata_sample.csv
 master.csv
 MAY.xlsx
 MAY.xlsx.gsheet

In [7]:
cd Hatespeech-data

/content/drive/My Drive/Hatespeech-data


In [8]:
ls

bidirectionallstm.h5  test.csv  train.csv


In [0]:
datapathtrain=(os.getcwd())+"/train.csv"
datapathtest=(os.getcwd())+"/test.csv"

In [10]:
import re
import nltk
from nltk.corpus import stopwords
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers.convolutional import Conv1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, LSTM, Embedding
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Embedding, Input

Using TensorFlow backend.


In [20]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from sklearn.utils import shuffle
train=(pd.read_csv(datapathtrain,encoding='latin-1'))
test=(pd.read_csv(datapathtest,encoding='latin-1'))

In [13]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [29]:
cd ..


/content/drive/My Drive


In [30]:
ls

 1stone.h5
 20151122_223055.mp4
'AAA-Literature reviews:.gdoc'
 AAARBI.gdoc
'AAA SOP Progress.gdoc'
'AAA stopwords.gdoc'
'AA BAV Project.gdoc'
'Aa bavv2.gdoc'
 ABC.xlsx
 ans.csv
 ASD.gdoc
 bidirectional42i.h5
 bidirectionalffhai.h5
 bidirectional.h5
 bidirectionalhai.h5
 bidirectionalwithoutglobal.h5
 CNN_Data/
'Colab Notebooks'/
 companies.docx
'Copy of glove.twitter.27B.100d.txt'
'Copy of glove.twitter.27B.50d.txt'
 datarnn.csv
 DESCRIPTION.docx
 DESCRIPTION.gdoc
'dna structure1.ppt'
'DRM Ass 2.2.xlsx'
'DRM Ass 2.2.xlsx.gsheet'
 dt/
 GAIL_Group37_Futures.xlsx
 GAIL_Group37_Futures.xlsx.gsheet
'GAIL_Group37_Underlying Asset.xlsx'
'GAIL_Group37_Underlying Asset.xlsx.gsheet'
 GANs.zip
 GroceryProcessed.csv
 Hatespeech-data/
'Ian Goodfellow, Yoshua Bengio, Aaron Courville-Deep Learning-MIT Press (2016).pdf'
'Ideas and constrains of the strategies.gdoc'
 inc_occ_gender.csv
 instructions_1.pdf
'log returns.xlsx'
 lstmheading.csv
 marketdata_sample.csv
 master.csv
 MAY.xlsx
 MAY.xlsx.gsheet

In [0]:
embeddings_index = dict()
f = open('Copy of glove.twitter.27B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()


In [37]:
embeddings_index['a']

array([ 0.86323  ,  0.031356 ,  0.10169  ,  0.26639  ,  0.19313  ,
       -0.076727 , -0.22647  , -0.69596  , -0.63946  , -0.8632   ,
       -0.29465  , -0.31175  , -4.4257   , -0.16769  ,  0.23197  ,
       -0.0085179, -0.063032 , -0.044064 , -0.23138  ,  0.59465  ,
       -0.1334   , -0.61637  , -0.019008 , -0.31235  , -0.2403   ,
       -3.112    ,  0.22267  , -0.046524 , -0.046095 ,  1.1434   ,
        0.60818  ,  0.34767  ,  0.36155  ,  0.35258  , -0.16617  ,
        0.82837  ,  0.35088  , -0.23608  , -0.25425  ,  0.55587  ,
       -1.4276   ,  0.06918  ,  0.015027 , -0.45487  ,  0.63978  ,
       -0.16407  ,  0.14985  ,  0.94771  ,  0.23274  , -0.51445  ,
        0.70982  ,  0.60018  ,  0.047234 , -0.39084  , -0.14794  ,
        0.68263  , -0.12995  , -0.22846  ,  0.43185  , -0.10681  ,
        0.06544  ,  0.34506  ,  0.089428 ,  0.19983  ,  1.1775   ,
       -0.33236  , -0.60181  ,  0.38324  , -0.090755 , -0.15759  ,
       -0.23093  , -0.88441  ,  0.07837  ,  0.19774  , -0.1060

In [0]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_indexs = dict(get_coefs(*o.strip().split()) for o in open('Copy of glove.twitter.27B.100d.txt'))


ValueError: ignored

In [0]:
classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

In [0]:
sentences_train = train["comment_text"].fillna("na").values

In [0]:
y = train[classes].values

In [0]:
sentences_test = test["comment_text"].fillna("na").values

In [0]:
max_features = 50000
maxlen = 200
listofstopwords = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize 


In [27]:
print(listofstopwords)

{'each', 'ain', 's', 'having', 'are', 'out', "it's", 'does', 'd', 'aren', 'an', "you'd", 'ma', 'has', 'were', 'had', 'own', 'then', 'again', "couldn't", 'she', 'where', 'can', 'couldn', 'down', 'wouldn', 're', 'below', 'will', 'our', 'him', 'them', 'needn', 'mustn', 'his', 'won', 'if', 'who', 've', 'most', 'been', 'you', 'that', 'theirs', 'not', 'during', 'more', 't', "mustn't", 'for', "aren't", "didn't", 'hasn', 'here', 'itself', 'didn', 'nor', 'i', 'and', 'than', 'he', 'now', 'myself', 'hers', 'above', 'on', 'as', 'a', 'yourselves', 'wasn', 'once', 'some', 'themselves', 'should', 'into', 'ours', "shan't", 'doesn', 'when', 'very', 'from', 'up', 'll', 'isn', 'about', "weren't", 'was', 'how', "should've", 'there', 'yourself', 'their', 'between', 'so', 'but', 'because', 'in', 'off', 'hadn', "wouldn't", 'this', 'shan', 'himself', 'weren', 'shouldn', 'what', "don't", 'no', 'until', "wasn't", 'y', 'have', "that'll", 'other', 'm', 'any', 'me', "hadn't", 'only', "doesn't", 'being', 'did', 'he

In [0]:
#
#Removing punctuations
#Lowering Words 
#Removing Non Alphabets
#Removing Stopwords

In [0]:
for x in range(len(sentences_train)):
    #sentences_train[x]=sentences_train[x].to_lower()
    sentences_train[x] =sentences_train[x].replace('\{(.*?)\}','')
    sentences_train[x] = re.sub('[^A-Za-z]+', ' ', sentences_train[x])
    word_tokens = word_tokenize(sentences_train[x])
    filtered_sentence = [w.lower() for w in word_tokens if not w in listofstopwords]
    sentences_train[x] = " ".join(w for w in filtered_sentence)
for x in range(len(sentences_test)):
    #sentences_test[x]=sentences_test[x].to_lower()
    sentences_test[x] =sentences_test[x].replace('\{(.*?)\}','')
    sentences_test[x] = re.sub('[^A-Za-z]+', ' ', sentences_test[x])
    word_tokens = word_tokenize(sentences_test[x])
    filtered_sentence = [w.lower() for w in word_tokens if not w in listofstopwords]
    sentences_test[x] = " ".join(w for w in filtered_sentence)

In [26]:
sentences_train[0]

'explanation why edits made username hardcore metallica fan reverted they vandalisms closure gas i voted new york dolls fac and please remove template talk page since i retired'

In [0]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(sentences_train))
tokenized_train = tokenizer.texts_to_sequences(sentences_train)
tokenized_test = tokenizer.texts_to_sequences(sentences_test)
X_train = pad_sequences(tokenized_train, maxlen=maxlen)
X_test = pad_sequences(tokenized_test, maxlen=maxlen)

In [41]:
word_index = tokenizer.word_index
print(len(word_index))

168809


In [0]:
vocab_size=min(max_features,len(word_index))

In [0]:
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in word_index.items():
    if i >= max_features:
      continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector

In [0]:
checkpoint = ModelCheckpoint('bidirectionallstmwithpreprocessing.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

In [0]:
early = EarlyStopping(monitor="val_acc", mode="auto", patience=3)

In [0]:
callbacks_list = [checkpoint, early] 

In [0]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, 100, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.2))(x)
x = GlobalMaxPool1D()(x)
x = Dense(64, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
epochs=5
batch_size=1024

In [76]:
model.fit(X_train, y, batch_size=batch_size,epochs=epochs, validation_split=0.2, callbacks=callbacks_list)

Train on 127656 samples, validate on 31915 samples
Epoch 1/5
127656/127656 [==============================] - 167s 1ms/step - loss: 0.1729 - acc: 0.9501 - val_loss: 0.0598 - val_acc: 0.9782

Epoch 00001: val_acc improved from -inf to 0.97820, saving model to bidirectionallstmwithpreprocessing.h5
Epoch 2/5
127656/127656 [==============================] - 164s 1ms/step - loss: 0.0606 - acc: 0.9792 - val_loss: 0.0494 - val_acc: 0.9817

Epoch 00002: val_acc improved from 0.97820 to 0.98170, saving model to bidirectionallstmwithpreprocessing.h5
Epoch 3/5
127656/127656 [==============================] - 163s 1ms/step - loss: 0.0519 - acc: 0.9814 - val_loss: 0.0474 - val_acc: 0.9822

Epoch 00003: val_acc improved from 0.98170 to 0.98223, saving model to bidirectionallstmwithpreprocessing.h5
Epoch 4/5
127656/127656 [==============================] - 162s 1ms/step - loss: 0.0482 - acc: 0.9821 - val_loss: 0.0459 - val_acc: 0.9825

Epoch 00004: val_acc improved from 0.98223 to 0.98248, saving mod

In [0]:
early = EarlyStopping(monitor="val_loss", mode="auto", patience=3)

In [0]:
callbacks_list = [checkpoint, early] 